In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools # for itertools.repeat - repeating the years N times
from string import digits # for removing trailing digits

import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
os.chdir('/Users/apple/Desktop/ResearchProject/New Data/IMF20102020')

In [3]:
exports = pd.read_csv("exports4.csv")
# to do the same handling for exports
# to change the yearmonth formats into yyyymm
# 1. break into YYYY and MXX and do the handling
exports['year'] = exports['yearmonth'].str[:4]
exports['month'] = exports['yearmonth'].str[5:].apply(lambda x: x.zfill(2))

# concat and handle the 2 digit format to create a new yearmonth 
exports['year_month'] = exports['year'] + exports['month']

# drop unnamed: 0 column and yearmonth column
exports = exports.drop(columns=['Unnamed: 0', 'yearmonth'])
exports.head()

,country_name,partner_name,exports,year,month,year_month
0,Albania,Japan,0.033463,2010,01,201001
1,Argentina,Bulgaria,0.709131,2010,01,201001
2,Argentina,Bosnia and Herzegovina,0.464390,2010,01,201001
3,Argentina,Barbados,0.211196,2010,01,201001
4,Algeria,Cuba,14.816428,2010,01,201001


In [4]:
# subset 2010 - 2020 only to see any year on year / months patterns (seasonality)
exports10_20 = exports[exports['year'] != '2021']
exports10_20['year']

0          2010
1          2010
2          2010
3          2010
4          2010
           ... 
3410743    2020
3410744    2020
3410745    2020
3410746    2020
3410747    2020
Name: year, Length: 3410748, dtype: object

In [5]:
exports_by_year = exports10_20.groupby(['year']).sum().reset_index()
exports_by_year.head()

,year,exports
0,2010,1.390830e+07
1,2011,1.659379e+07
2,2012,1.676907e+07
3,2013,1.701988e+07
4,2014,1.715377e+07


In [6]:
exports_by_year = exports10_20.groupby(['year']).sum().reset_index()
exports_by_year.head()

,year,exports
0,2010,1.390830e+07
1,2011,1.659379e+07
2,2012,1.676907e+07
3,2013,1.701988e+07
4,2014,1.715377e+07


In [7]:
# to explore the countries with top exports/imports
exports_by_country_year = exports10_20.groupby(['country_name', 'year']).sum().reset_index()
exports_by_country_year.head()

,country_name,year,exports
0,"Afghanistan, Islamic Rep. of",2010,388.300915
1,"Afghanistan, Islamic Rep. of",2011,374.938418
2,"Afghanistan, Islamic Rep. of",2012,305.795136
3,"Afghanistan, Islamic Rep. of",2013,373.958202
4,"Afghanistan, Islamic Rep. of",2014,414.759213


In [8]:
# get quantile to get a sense of data
exports_total_by_year = exports_by_country_year.sort_values(["year", "exports"], ascending=[True, False]).groupby("year").head(177)
np.percentile(exports_total_by_year['exports'], [24.7, 75, 80.5], axis =0)

array([  999.24148593, 59709.0537225 , 99637.76217485])

In [9]:
# can cut off at 1,000 and 100,000
# subset countries to see 
bins = [-1, 1000, 10000, 100000, 100000000]
category = ['low', 'mid', 'high', 'mega']
exports_total_by_year['clus'] = pd.cut(exports_total_by_year['exports'], bins, labels=category)
exports_total_by_year

,country_name,year,exports,clus
363,"China, P.R.: Mainland",2010,1.343261e+06,mega
1848,United States,2010,1.241198e+06,mega
616,Germany,2010,1.234731e+06,mega
836,Japan,2010,7.233241e+05,mega
1199,"Netherlands, The",2010,5.625644e+05,mega
...,...,...,...,...
604,"Gambia, The",2020,2.745013e+01,low
1924,"Yemen, Rep. of",2020,2.730006e+01,low
659,Grenada,2020,1.264680e+01,low
1308,"Palau, Rep. of",2020,2.724716e+00,low


In [10]:
exports_total_by_year.groupby(['year', 'clus'])['country_name'].count()
# seems not that different

year  clus
2010  low     45
      mid     58
      high    42
      mega    32
2011  low     43
      mid     54
      high    43
      mega    37
2012  low     42
      mid     56
      high    43
      mega    36
2013  low     41
      mid     54
      high    47
      mega    35
2014  low     39
      mid     54
      high    48
      mega    36
2015  low     45
      mid     55
      high    44
      mega    33
2016  low     46
      mid     55
      high    44
      mega    32
2017  low     44
      mid     54
      high    46
      mega    33
2018  low     45
      mid     52
      high    45
      mega    35
2019  low     47
      mid     49
      high    46
      mega    35
2020  low     44
      mid     55
      high    44
      mega    34
Name: country_name, dtype: int64

In [11]:
xp_df = exports_total_by_year.sort_values(by=['country_name', 'year'])
xp_df

,country_name,year,exports,clus
0,"Afghanistan, Islamic Rep. of",2010,388.300915,low
1,"Afghanistan, Islamic Rep. of",2011,374.938418,low
2,"Afghanistan, Islamic Rep. of",2012,305.795136,low
3,"Afghanistan, Islamic Rep. of",2013,373.958202,low
4,"Afghanistan, Islamic Rep. of",2014,414.759213,low
...,...,...,...,...
1942,Zimbabwe,2016,2584.330370,mid
1943,Zimbabwe,2017,2587.577109,mid
1944,Zimbabwe,2018,3046.957147,mid
1945,Zimbabwe,2019,3176.852941,mid


In [12]:
# note that the ex1920['clus'] needs to add .astype("str") because it was not str type but category type
xp_df['state'] = xp_df['country_name'] + "-" + xp_df['clus'].astype("str")

In [13]:
# to loop in the countries then as a whole
country_list = xp_df['country_name'].unique()

In [14]:
def changes(i):
   # for i in range(0, len(country_list)):
        cty = xp_df[xp_df['country_name'] ==country_list[i]]

        comp_cty=cty['state'].reset_index()

        # to mark overlap for the same room -> true or false
        same = []
        for i in range(1, len(comp_cty)):
            if (comp_cty['state'][i-1] == comp_cty['state'][i]):
                var = True
            else:
                var = False
            same.append(var)
        len(same)

        same.append(True)

        var = []
        same_ind = same[:]
        # make it reversed logic i.e. if i is F then i+1 is F
        for i in range(0, 10):
            if same[i] == False:
                same_ind[i+1] = False 
            else: True
            same_ind.append(var)

        # to add the logic for the final row
        mapped_change = same_ind[0:11]

        cty['same'] = mapped_change

        change = cty[cty['same']==False]
        return change

In [15]:
# loop to append in a bigger data frame
all_change = pd.DataFrame()
for i in range(0, len(country_list)):
    changed = changes(i)
    all_change = all_change.append(changed,ignore_index=True)

In [16]:
all_change

,country_name,year,exports,clus,state,same
0,"Bahamas, The",2013,844.218561,low,"Bahamas, The-low",False
1,"Bahamas, The",2014,1863.336403,mid,"Bahamas, The-mid",False
2,"Bahamas, The",2015,662.677180,low,"Bahamas, The-low",False
3,"Bahamas, The",2017,465.713200,low,"Bahamas, The-low",False
4,"Bahamas, The",2018,1109.758440,mid,"Bahamas, The-mid",False
...,...,...,...,...,...,...
149,"Venezuela, Rep. Bolivariana de",2020,5556.767779,mid,"Venezuela, Rep. Bolivariana de-mid",False
150,Vietnam,2011,90602.898920,high,Vietnam-high,False
151,Vietnam,2012,106658.148200,mega,Vietnam-mega,False
152,"Yemen, Rep. of",2014,2460.815178,mid,"Yemen, Rep. of-mid",False


In [45]:
# to loop in the years then as a whole
year_list = xp_df['year'].unique()

In [46]:
year_list

array(['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019', '2020'], dtype=object)

In [60]:
# the logic of the loop is that we subset the countries with changes in the previous year and current year and merge
# to get the pairs and composite the status e.g. low to high, high to med
change_flow = pd.DataFrame()
for i in range(0, (len(year_list)-1)):
    year_p = all_change[all_change['year']==year_list[i]][['country_name', 'year','clus']]
    year_c = all_change[all_change['year']==year_list[i+1]][['country_name', 'year','clus']]
    change = year_p.merge(year_c, on='country_name', how='left')
    change[['change']] = change['clus_x'].astype("str") + ' to ' + change['clus_y'].astype("str")
    change_flow = change_flow.append(change,ignore_index=True)

In [65]:
# only 2010 can have full match as the following years, part of the change are from previous year and 
# part are from next years. thus need to filter out the rows that change is null
change_flow_full = change_flow[change_flow['change'].notnull()]
change_flow_full

,country_name,year_x,clus_x,year_y,clus_y,change
0,Brunei Darussalam,2010,mid,2011,high,mid to high
1,"Congo, Rep. of",2010,mid,2011,high,mid to high
2,Denmark,2010,high,2011,mega,high to mega
3,"Equatorial Guinea, Rep. of",2010,mid,2011,high,mid to high
4,Gabon,2010,mid,2011,high,mid to high
...,...,...,...,...,...,...
138,Rwanda,2019,low,2020,mid,low to mid
139,Sierra Leone,2019,low,2020,mid,low to mid
140,"South Sudan, Rep. of",2019,mid,2020,low,mid to low
141,Turkmenistan,2019,high,2020,mid,high to mid


In [77]:
# also need to remove the cases that the two years are merged but actually no change
# only due to changes as compared to previous and next year
change_flow_full['no_change'] = (change_flow_full['clus_x'] == change_flow_full['clus_y'])

In [127]:
change_flow_final = change_flow_full[change_flow_full['no_change']==False][['country_name', 'year_x', 'year_y', 'change']]

In [128]:
change_flow_final

,country_name,year_x,year_y,change
0,Brunei Darussalam,2010,2011,mid to high
1,"Congo, Rep. of",2010,2011,mid to high
2,Denmark,2010,2011,high to mega
3,"Equatorial Guinea, Rep. of",2010,2011,mid to high
4,Gabon,2010,2011,mid to high
...,...,...,...,...
138,Rwanda,2019,2020,low to mid
139,Sierra Leone,2019,2020,low to mid
140,"South Sudan, Rep. of",2019,2020,mid to low
141,Turkmenistan,2019,2020,high to mid


In [84]:
change_flow_final[change_flow_final['year_x']=='2010'][['change']].value_counts().sort_index()

change      
high to mega    5
high to mid     1
low to mid      2
mid to high     7
dtype: int64

In [85]:
change_flow_final[change_flow_final['year_x']=='2011'][['change']].value_counts().sort_index()

change      
high to mega    1
high to mid     3
low to mid      1
mega to high    2
mid to high     2
dtype: int64

In [86]:
change_flow_final[change_flow_final['year_x']=='2012'][['change']].value_counts().sort_index()

change      
low to mid      1
mega to high    1
mid to high     3
dtype: int64

In [90]:
change_flow_final[change_flow_final['year_x']=='2013'][['change']].value_counts().sort_index()

change      
high to mega    1
low to mid      4
mid to high     2
mid to low      2
dtype: int64

In [91]:
change_flow_final[change_flow_final['year_x']=='2014'][['change']].value_counts().sort_index()

change      
high to mid     7
mega to high    3
mid to low      6
dtype: int64

In [92]:
change_flow_final[change_flow_final['year_x']=='2015'][['change']].value_counts().sort_index()

change      
high to mid     1
mega to high    1
mid to low      1
dtype: int64

In [93]:
change_flow_final[change_flow_final['year_x']=='2016'][['change']].value_counts().sort_index()

change      
high to mega    1
low to mid      2
mid to high     3
dtype: int64

In [94]:
change_flow_final[change_flow_final['year_x']=='2017'][['change']].value_counts().sort_index()

change      
high to mega    2
low to mid      1
mid to high     1
mid to low      2
dtype: int64

In [95]:
change_flow_final[change_flow_final['year_x']=='2018'][['change']].value_counts().sort_index()

change     
high to mid    1
mid to high    2
mid to low     2
dtype: int64

In [96]:
change_flow_final[change_flow_final['year_x']=='2019'][['change']].value_counts().sort_index()

change      
high to mid     3
low to mid      5
mega to high    1
mid to low      2
dtype: int64

In [116]:
# not much change year on year 
# how about 2010 vs. 2020
trade_10_20 = xp_df[xp_df['year']=='2010'].merge(xp_df[xp_df['year']=='2020'], on= 'country_name')
trade_10_20

,country_name,year_x,exports_x,clus_x,state_x,year_y,exports_y,clus_y,state_y
0,"Afghanistan, Islamic Rep. of",2010,388.300915,low,"Afghanistan, Islamic Rep. of-low",2020,776.515384,low,"Afghanistan, Islamic Rep. of-low"
1,Albania,2010,1517.178041,mid,Albania-mid,2020,2636.041033,mid,Albania-mid
2,Algeria,2010,56996.694801,high,Algeria-high,2020,25980.860185,high,Algeria-high
3,Angola,2010,49473.721076,high,Angola-high,2020,13045.113729,high,Angola-high
4,Antigua and Barbuda,2010,123.927877,low,Antigua and Barbuda-low,2020,181.549598,low,Antigua and Barbuda-low
...,...,...,...,...,...,...,...,...,...
172,"Venezuela, Rep. Bolivariana de",2010,62133.445371,high,"Venezuela, Rep. Bolivariana de-high",2020,5556.767779,mid,"Venezuela, Rep. Bolivariana de-mid"
173,Vietnam,2010,68340.539085,high,Vietnam-high,2020,266553.933691,mega,Vietnam-mega
174,"Yemen, Rep. of",2010,6330.228764,mid,"Yemen, Rep. of-mid",2020,27.300056,low,"Yemen, Rep. of-low"
175,Zambia,2010,6841.245376,mid,Zambia-mid,2020,6761.884231,mid,Zambia-mid


In [117]:
trade_10_20['no_change'] = (trade_10_20['clus_x'] == trade_10_20['clus_y'])

In [119]:
trade_10_20_change = trade_10_20[trade_10_20['no_change']==False]
trade_10_20_change[['change']] = trade_10_20_change['clus_x'].astype("str") + ' to ' + trade_10_20_change['clus_y'].astype("str")
trade_10_20_change

,country_name,year_x,exports_x,clus_x,state_x,year_y,exports_y,clus_y,state_y,no_change,change
17,Benin,2010,519.848983,low,Benin-low,2020,1138.176589,mid,Benin-mid,False,low to mid
27,Cambodia,2010,4184.982245,mid,Cambodia-mid,2020,14217.339607,high,Cambodia-high,False,mid to high
36,Costa Rica,2010,8961.643020,mid,Costa Rica-mid,2020,11577.724504,high,Costa Rica-high,False,mid to high
41,Denmark,2010,93367.698627,high,Denmark-high,2020,105414.121952,mega,Denmark-mega,False,high to mega
57,Ghana,2010,5202.256211,mid,Ghana-mid,2020,17908.293024,high,Ghana-high,False,mid to high
60,Guatemala,2010,8416.538473,mid,Guatemala-mid,2020,11412.884615,high,Guatemala-high,False,mid to high
63,Guyana,2010,891.217579,low,Guyana-low,2020,2390.324591,mid,Guyana-mid,False,low to mid
66,Hungary,2010,88924.131981,high,Hungary-high,2020,111346.914001,mega,Hungary-mega,False,high to mega
81,"Korea, Dem. People's Rep. of",2010,2127.028392,mid,"Korea, Dem. People's Rep. of-mid",2020,188.534391,low,"Korea, Dem. People's Rep. of-low",False,mid to low
85,Latvia,2010,9429.858102,mid,Latvia-mid,2020,16038.759109,high,Latvia-high,False,mid to high


## Important patterns: 
### Syrian Arab Rep. slumped from high to low (2 levels) in 2011. Decreased from 44k to <1k from 2010 to 2014
### North Korea droped from high to mid in 2017（UN sanctions on North Korea in 2017）
### Vietnam moved from high to mega with steady increase during the last 10 years, even during covid year 2020
### Cambodia and Ghana improved from mid to high (more than doubled)
### Guyana moved from low to high (country with good economic growth since late 2010s)

In [120]:
trade_10_20_change[trade_10_20_change['year_x']=='2010'][['change']].value_counts().sort_index()

change      
high to low     1
high to mega    3
high to mid     2
low to mid      6
mega to high    1
mid to high     7
mid to low      4
dtype: int64

In [130]:
# xp_df[xp_df['country_name']=='Syrian Arab Rep.']

In [134]:
#xp_df[xp_df['country_name']=="Korea, Dem. People's Rep. of"]

In [136]:
# xp_df[xp_df['country_name']=="Norway"]

In [138]:
# xp_df[xp_df['country_name']=="Vietnam"]

In [139]:
# big drop in 2018
# xp_df[xp_df['country_name']=="Liberia"]

,country_name,year,exports,clus,state
957,Liberia,2010,1377.315910,mid,Liberia-mid
958,Liberia,2011,1118.296928,mid,Liberia-mid
959,Liberia,2012,1131.959377,mid,Liberia-mid
960,Liberia,2013,1568.227364,mid,Liberia-mid
961,Liberia,2014,1626.754653,mid,Liberia-mid
962,Liberia,2015,1422.481721,mid,Liberia-mid
963,Liberia,2016,1155.459389,mid,Liberia-mid
964,Liberia,2017,1468.864175,mid,Liberia-mid
965,Liberia,2018,457.103854,low,Liberia-low
966,Liberia,2019,500.805043,low,Liberia-low
